<a href="https://colab.research.google.com/github/rohithr2511/FaceTextSummarization/blob/main/Copy_of_FaceTextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# necessary libraries
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
import os

In [2]:
# input and output folders
input_dir = "samples"
output_dir = "samples"
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

In [3]:
# Function to load summarization pipeline
def load_summarizer():
    model_name = "facebook/bart-large-cnn"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    return pipeline("summarization", model=model, tokenizer=tokenizer)

In [4]:
# Function to chunk long text into smaller parts
def chunk_text(text, max_tokens=800):
    words = text.split()
    for i in range(0, len(words), max_tokens):
        yield " ".join(words[i:i + max_tokens])

In [5]:
# summarizing one document
def summarize(text, summarizer):
    chunks = list(chunk_text(text))
    summary = ""
    for chunk in tqdm(chunks, desc="Summarizing"):
        result = summarizer(chunk, max_length=200, min_length=100, do_sample=False)[0]
        summary += result['summary_text'] + " "
    return summary.strip()

In [6]:
# loading model once
summarizer = load_summarizer()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Device set to use cpu


In [7]:
# All input files Calling
input_files = sorted([f for f in os.listdir(input_dir) if f.startswith("input") and f.endswith(".txt")])

In [8]:
# looping through files and generate summaries
for input_file in input_files:
    input_path = os.path.join(input_dir, input_file)
    number = input_file.replace("input", "").replace(".txt", "")
    output_file = f"summary{number}.txt"
    output_path = os.path.join(output_dir, output_file)

    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()

    print(f"🔄 Processing {input_file}...")
    summary = summarize(text, summarizer)

    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(summary)

    print(f"✅ Saved summary to {output_file}\n")

print("🎉 All summaries complete!")

🎉 All summaries complete!
